#Importando Bibliotecas

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import tokenize 
from nltk.stem import RSLPStemmer
from nltk.stem.porter import *
import heapq
import heapq as hp
import time
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


##Lendo Arquivo CSV

In [0]:
arquivo = pd.read_csv('results.csv')
texto = arquivo['text']

def eliminar_carac(text):
  return re.sub(r'(\W+|\d+|\b\w{0,2}\b)', ' ', text.lower())
texto = texto.apply(eliminar_carac)

texto = texto.apply(tokenize.word_tokenize, language='portuguese')
arquivo.head()

In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def eliminar_stopwords(text):
  palavras = []
  for palavra in text:
    if(palavra not in stopwords):
      palavras.append(palavra)
  return palavras 

def build_index(arquivo):
  numero_da_lista = 0
  dic_index = {}
  
  for documento in arquivo:
    
    numero_da_lista += 1
    doc = eliminar_stopwords(documento)
    
    for palavra in doc:
      ocorrencia = doc.count(palavra)
      par = (numero_da_lista, ocorrencia)
      if palavra not in dic_index:
        dic_index[palavra] = []
      
      if par not in dic_index[palavra]:
         dic_index[palavra].append(par)
  return dic_index
index = {}              
index = build_index(texto)

In [0]:
M = texto.count()
IDF = {}
for pa in index:
  k = len(index[pa])
  idf = round(np.log((M+1)/k),2)
  IDF[pa] = idf

## Questão 1 :

In [0]:
numero_doc = 200
document = arquivo.loc[numero_doc]
query = 'ditadura militar'

document.title

'O ‘Abaporu’ visita São Paulo'

## Modelos vetorias do lab anterior

In [0]:
def binario_r(query, doc):
  contador = 0
  query_palavras = query.split()
  doc_palavras = doc.split()
  
  for palavra in query_palavras:
    contador += (palavra in doc_palavras)
    
  return contador

def tf_fun(query, doc):
  contador = 0
  doc_palavras = doc.split()
  query_palavras = query.split()
  
  for palavra in query_palavras:
    contador += doc_palavras.count(palavra)
  
  return contador

def tf_idf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in index:
      score += cwd * len(index[word][-1])
  
  return round(score,2)


def bm25(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in index:
      dfw = len(index[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(score,2)

In [0]:
import bisect
def topX(query,tamanho):
  n = 0
  binario = []
  tf = []
  tfidf = []
  b25 = []
  for doc in arquivo.text:
    doc = doc.lower()
    n += 1
    bisect.insort(binario, (binario_r(query, doc), n))
    bisect.insort(tf, (tf_fun(query, doc), n))
    bisect.insort(tfidf, (tf_idf(query, doc), n))
    bisect.insort(b25, (bm25(query, doc,20), n))
 
  binario.reverse()
  tf.reverse()
  tfidf.reverse()
  b25.reverse()
  
  return binario[:tamanho],tf[:tamanho],tfidf[:tamanho],b25[:tamanho]

In [0]:
topX_b, topX_tf, topX_tfidf, topX_bm25 = topX(query,10)
df = pd.DataFrame()


df['Binary'] = topX_b
df['TF'] = topX_tf
df['TF IDF'] = topX_tfidf
df['BM25'] = topX_bm25
df

,Binary,TF,TF IDF,BM25
0,"(2, 238)","(22, 7)","(44, 7)","(12.54, 7)"
1,"(2, 233)","(15, 238)","(30, 238)","(9.45, 238)"
2,"(2, 230)","(13, 115)","(26, 115)","(8.0, 115)"
3,"(2, 229)","(12, 25)","(24, 25)","(7.9, 3)"
4,"(2, 224)","(12, 3)","(24, 3)","(7.67, 216)"
5,"(2, 223)","(11, 223)","(22, 223)","(7.56, 172)"
6,"(2, 218)","(11, 216)","(22, 216)","(7.23, 223)"
7,"(2, 216)","(11, 172)","(22, 172)","(7.14, 95)"
8,"(2, 208)","(10, 95)","(20, 95)","(6.83, 25)"
9,"(2, 172)","(7, 169)","(14, 169)","(5.36, 6)"


In [0]:
def reciprocal_R(tuples, docId):
  n = 1.0;
  for r,doc in tuples:
    if doc == docId:
      return  [round(1 / n, 2)]
    else:
      n += 1

reciprocal_rank_df = pd.DataFrame()
reciprocal_rank_df['Binary'] = reciprocal_R(df['Binary'], numero_doc)
reciprocal_rank_df['TF'] = reciprocal_R(df['TF'], numero_doc)
reciprocal_rank_df['TF IDF'] = reciprocal_R(df['TF IDF'], numero_doc)
reciprocal_rank_df['BM25'] = reciprocal_R(df['BM25'], numero_doc)
rank_df

,Binary,TF,TF IDF,BM25
1,1.0,None,0.25,0.33


# Questão 2

##Lendo Arquivo JSON

In [0]:
arquivo_json = pd.read_json('results_final.json')

feedback = {arquivo_json['query'][i]:arquivo_json['docs'][i] for i in range(10)}


In [0]:
def intersection(a,b):
  return [elemento for elemento in a if elemento in b]

def doc_indexes(model):
  return [documento for pontuacao,documento in model]

def tamanho(elemento):
  return len(elemento)

def ap(query):
  docs = []

  for doc_info in feedback[query]:
    row = arquivo.loc[arquivo.url == doc_info['URL']]
    docs.append(row.index[0])
  
  binary, tf, tfidf, bm25 = topX(query, 10)
  binary= doc_indexes(binary)
  tf= doc_indexes(tf)
  tfidf=doc_indexes(tfidf)
  bm25=doc_indexes(bm25)
 
  
  calculo_ap_binary = len(intersection(binary, docs)) / tamanho(binary)
  calculo_ap_tf = len(intersection(tf, docs)) / tamanho(tf)
  calculo_ap_tfidf = len(intersection(tfidf, docs)) / tamanho(tfidf)
  calculo_ap_bm25 = len(intersection(bm25, docs)) / tamanho(bm25)
  
  return  calculo_ap_binary,  calculo_ap_tf,  calculo_ap_tfidf,  calculo_ap_bm25
  

## Calculo Do MAP

In [0]:
def map_Calculo(queries):
  binary_num = 0
  tf_num = 0
  tfidf_num = 0
  bm25_num = 0
  
  for query in queries:
    binary_Ap, tf_Ap, tfidf_Ap, bm25_Ap = ap(query)
    binary_num =binary_num+ binary_Ap
    tf_num =tf_num+ tf_Ap
    tfidf_num = tfidf_num+ tfidf_Ap
    bm25_num = bm25_num + bm25_Ap
  
  map_binary = round(binary_num / len(queries),2)
  map_tf = round(tf_num / len(queries),2)
  map_tfidf = round(tfidf_num / len(queries),2)
  map_bm25 = round(bm25_num / len(queries),2)
  
  return map_binary, map_tf, map_tfidf, map_bm25



## Criação da Tabela Com tamanho 10 , do metodo TopX

In [0]:
map_binary, map_tf, map_tfidf, map_bm25 = map_Calculo(arquivo_json['query'])

map_df = pd.DataFrame()
map_df['Binary'] = [map_binary]
map_df['TF'] = [map_tf]
map_df['TF-IDF'] = [map_tfidf]
map_df['BM25'] = [map_bm25]
map_df


,Binary,TF,TF-IDF,BM25
0,0.03,0.0,0.01,0.01


## Binary obteve o melhor resultado, Maior valor do metodo map_Calculo

## Questão 3 avaliação multi-nível DCG